In [0]:
# Mount the Azure Blob Storage
dbutils.fs.mount(
    source = "wasbs://raw@aipoweredinsights.blob.core.windows.net",
    mount_point = "/mnt/raw2",
    extra_configs = {"fs.azure.account.key.aipoweredinsights.blob.core.windows.net":"enter-your-key"}
)

Out[2]: True

In [0]:
# Listing content
dbutils.fs.ls("/mnt/raw2/")

Out[4]: [FileInfo(path='dbfs:/mnt/raw2/dbo.ai_job_market_insights.txt', name='dbo.ai_job_market_insights.txt', size=54485, modificationTime=1726382432000)]

In [0]:
# Create a dataframe
df = spark.read.format("csv").options(header='True', inferSchema='True').load('dbfs:/mnt/raw2/dbo.ai_job_market_insights.txt')

In [0]:
# View content
display(df)

Job_Title,Industry,Company_Size,Location,AI_Adoption_Level,Automation_Risk,Required_Skills,Salary_USD,Remote_Friendly,Job_Growth_Projection
Cybersecurity Analyst,Entertainment,Small,Dubai,Medium,High,UX/UI Design,111392.1640625,true,Growth
Marketing Specialist,Technology,Large,Singapore,Medium,High,Marketing,93792.5625,false,Decline
AI Researcher,Technology,Large,Singapore,Medium,High,UX/UI Design,107170.265625,true,Growth
Sales Manager,Retail,Small,Berlin,Low,High,Project Management,93027.953125,false,Growth
Cybersecurity Analyst,Entertainment,Small,Tokyo,Low,Low,JavaScript,87752.921875,true,Decline
UX Designer,Education,Large,San Francisco,Medium,Medium,Cybersecurity,102825.0078125,false,Growth
HR Manager,Finance,Medium,Singapore,Low,High,Sales,102065.71875,true,Growth
Cybersecurity Analyst,Technology,Small,Dubai,Medium,Low,Machine Learning,86607.3203125,true,Decline
AI Researcher,Retail,Large,London,High,Low,JavaScript,75015.859375,false,Stable
Sales Manager,Entertainment,Medium,Singapore,High,Low,Cybersecurity,96834.578125,true,Decline


In [0]:
# Create table
df.createOrReplaceTempView("jobs")

In [0]:
%sql
SELECT *
FROM jobs
LIMIT 10;

Job_Title,Industry,Company_Size,Location,AI_Adoption_Level,Automation_Risk,Required_Skills,Salary_USD,Remote_Friendly,Job_Growth_Projection
Cybersecurity Analyst,Entertainment,Small,Dubai,Medium,High,UX/UI Design,111392.1640625,true,Growth
Marketing Specialist,Technology,Large,Singapore,Medium,High,Marketing,93792.5625,false,Decline
AI Researcher,Technology,Large,Singapore,Medium,High,UX/UI Design,107170.265625,true,Growth
Sales Manager,Retail,Small,Berlin,Low,High,Project Management,93027.953125,false,Growth
Cybersecurity Analyst,Entertainment,Small,Tokyo,Low,Low,JavaScript,87752.921875,true,Decline
UX Designer,Education,Large,San Francisco,Medium,Medium,Cybersecurity,102825.0078125,false,Growth
HR Manager,Finance,Medium,Singapore,Low,High,Sales,102065.71875,true,Growth
Cybersecurity Analyst,Technology,Small,Dubai,Medium,Low,Machine Learning,86607.3203125,true,Decline
AI Researcher,Retail,Large,London,High,Low,JavaScript,75015.859375,false,Stable
Sales Manager,Entertainment,Medium,Singapore,High,Low,Cybersecurity,96834.578125,true,Decline


# Transformations

## 1. Average Salary by Industry.

In [0]:
%sql
SELECT 
  Industry,
  AVG(Salary_USD) AS Average_Industry_Salary
FROM jobs
GROUP BY Industry
ORDER BY Average_Industry_Salary DESC;

Industry,Average_Industry_Salary
Finance,94355.46572818396
Entertainment,94291.23362699468
Education,93798.52186129386
Energy,92763.93949298469
Healthcare,91688.50409226191
Retail,91232.89733355978
Technology,91204.01157924107
Telecommunications,90418.30439268867
Manufacturing,86590.0035695043
Transportation,85058.29712540064


## 2. AI Adoption Rate per Industry.

In [0]:
%sql
SELECT Industry,
       AVG(CASE WHEN AI_Adoption_Level = 'High' THEN 1 ELSE 0 END) as High_AI_Adoption_Rate
FROM jobs
GROUP BY Industry
ORDER BY High_AI_Adoption_Rate DESC;

Industry,High_AI_Adoption_Rate
Healthcare,0.40476190476190477
Transportation,0.3076923076923077
Energy,0.30612244897959184
Retail,0.30434782608695654
Manufacturing,0.29310344827586204
Finance,0.2830188679245283
Telecommunications,0.2830188679245283
Education,0.2807017543859649
Entertainment,0.2765957446808511
Technology,0.23214285714285715


## 3. Automation Risk Distribution.

In [0]:
%sql
SELECT Automation_Risk, COUNT(*) as Job_Count
FROM jobs
GROUP BY Automation_Risk
ORDER BY Job_Count DESC;

Automation_Risk,Job_Count
Medium,173
High,169
Low,158


## 4. Remote Work Prevalence.

In [0]:
%sql
SELECT Industry,
       SUM(CASE WHEN Remote_friendly = TRUE THEN 1 ELSE 0 END)::FLOAT / COUNT(*) as Remote_Work_Rate
FROM jobs
GROUP BY Industry
ORDER BY Remote_Work_Rate DESC;

Industry,Remote_Work_Rate
Entertainment,0.5957446808510638
Healthcare,0.5714285714285714
Energy,0.5510204081632653
Transportation,0.5128205128205128
Finance,0.5094339622641509
Education,0.5087719298245614
Telecommunications,0.49056603773584906
Technology,0.44642857142857145
Retail,0.43478260869565216
Manufacturing,0.43103448275862066


## 5. Job Growth Projection by Industry.

In [0]:
%sql
SELECT Industry, Job_Growth_Projection
FROM jobs
GROUP BY Industry, Job_Growth_Projection
ORDER BY Job_Growth_Projection DESC;

Industry,Job_Growth_Projection
Healthcare,Stable
Telecommunications,Stable
Manufacturing,Stable
Entertainment,Stable
Transportation,Stable
Finance,Stable
Retail,Stable
Education,Stable
Technology,Stable
Energy,Stable


## 6. Skills Demand.

In [0]:
%sql
SELECT Industry, Required_Skills, COUNT(*) AS Total_Count
FROM jobs
GROUP BY Required_Skills, Industry
ORDER BY Total_Count DESC
LIMIT 10;

Industry,Required_Skills,Total_Count
Education,Project Management,10
Technology,Marketing,10
Education,Cybersecurity,9
Retail,Cybersecurity,9
Education,Data Analysis,9
Finance,Python,9
Telecommunications,Project Management,9
Technology,Sales,9
Energy,UX/UI Design,9
Manufacturing,Marketing,8


## 7. Salary Range for Different Company Sizes. 

In [0]:
%sql
SELECT Company_Size,
       MIN(Salary_USD) as Min_Salary,
       AVG(Salary_USD) as Avg_Salary,
       MAX(Salary_USD) as Max_Salary
FROM jobs
GROUP BY Company_Size
ORDER BY Avg_Salary DESC;

Company_Size,Min_Salary,Avg_Salary,Max_Salary
Small,31969.525390625,92165.97428956506,140476.0
Medium,41396.15625,90862.77511503067,155209.828125
Large,33601.3828125,90603.50315323795,138448.109375


## 8. Location-based Salary Variations.

In [0]:
%sql
SELECT Location,
       AVG(Salary_USD) as Avg_Salary,
       COUNT(*) as Job_Count
FROM jobs
GROUP BY Location
HAVING COUNT(*) > 10
ORDER BY Avg_Salary DESC;

Location,Avg_Salary,Job_Count
New York,93780.42637914541,49
Singapore,93739.80700231482,54
Berlin,93240.36889648438,48
Tokyo,92896.55392156863,51
Paris,92116.47359035326,46
Sydney,91885.40993088942,52
San Francisco,88953.23658014114,62
Toronto,88839.92902057926,41
London,88810.67263926631,46
Dubai,87892.45572916667,51


## 9. Correlation between AI Adoption and Salary.

In [0]:
%sql
SELECT AI_Adoption_Level,
       AVG(Salary_USD) as Avg_Salary,
       COUNT(*) as Job_Count
FROM jobs
GROUP BY AI_Adoption_Level
ORDER BY Avg_Salary DESC;

AI_Adoption_Level,Avg_Salary,Job_Count
Low,93353.59548985273,174
Medium,92139.14426937849,179
High,87583.42198129251,147


## 10. High-Growth Jobs with Low Automation Risk.

In [0]:
%sql
SELECT Job_title,
       AVG(Job_Growth_Projection) as Avg_Growth,
       AVG(CASE WHEN Automation_Risk = 'Low' THEN 1 ELSE 0 END) as Low_Risk_Rate
FROM jobs
GROUP BY Job_title
HAVING COUNT(*) > 5
ORDER BY Avg_Growth DESC, Low_Risk_Rate DESC
LIMIT 10;

Job_title,Avg_Growth,Low_Risk_Rate
Operations Manager,null,0.4090909090909091
Data Scientist,null,0.4032258064516129
Cybersecurity Analyst,null,0.38181818181818183
Sales Manager,null,0.3673469387755102
Software Engineer,null,0.3170731707317073
Product Manager,null,0.3076923076923077
AI Researcher,null,0.29411764705882354
HR Manager,null,0.2807017543859649
Marketing Specialist,null,0.20833333333333334
UX Designer,null,0.18518518518518517
